In [20]:
from dotenv import load_dotenv
import os
import pandas as pd
import matplotlib.pyplot as plt

from eth_rpc import PrivateKeyWallet
from emp_orderly.utils import from_address
from emp_orderly import (
    Strategy, EmpOrderly,
    crossover, plot_heatmaps,
    EMA, SMA, SLOPE, CHOP,
    EmpyrealOrderlySDK,
)
from emp_orderly.onboarding.faucet import request_testnet_funds as faucet
from emp_orderly.onboarding import deposit
from emp_orderly_types import PerpetualAssetType, Interval

load_dotenv()

True

In [21]:
wallet = PrivateKeyWallet.create_new()
orderly_id = from_address(wallet.address)

sdk = EmpyrealOrderlySDK(pvt_hex=wallet.private_key, account_id=orderly_id, is_testnet=True)

In [22]:
print(wallet.address)

0x031b39712C0735475a752ad73ca1d91c84DDE92C


In [23]:
print(await wallet.balance())

0


In [24]:
class EmaCross(Strategy):
    """Exponential Moving Average Death/Golden Cross Strategy.
    
    order_size: The amount of order that can be 
        made at a time.
    days_length_1: 12 days length for 
        calculating 12 days EMA.
    days_length_2: 26 days length for 
        calculating 26 days EMA.
    length: The number of days the Strategy is 
        intended to run for.
    """

    order_size: float = 0.999
    days_length_1: int = 12
    days_length_2: int = 26
    
    @classmethod
    def update_lags(cls, days_length_1, days_length_2):
        cls.days_length_1 = days_length_1
        cls.days_length_2 = days_length_2

    def init(self):
        close = self.data.close
        self.ema_days_1 = self.I(EMA, close, self.days_length_1)
        self.ema_days_2 = self.I(EMA, close, self.days_length_2)

    def next(self):
        if (
            crossover(self.ema_days_1, self.ema_days_2)
        ):
            self.position.close()
            self.buy(size=self.order_size)
        elif (
            crossover(self.ema_days_2, self.ema_days_1)
        ):
            self.position.close()
            self.sell(size=self.order_size)


tester = EmpOrderly(
    cash=1000000,
    commission=.0000001,
    exclusive_orders=True,
    sdk=sdk,
)

# load strategy and data
tester.set_strategy(EmaCross)
await tester.load_data(
    lookback=14,
    interval=Interval.five_minute,
    asset=PerpetualAssetType.DOGE,
)

# backtest
tester.backtest()

Start                     2024-09-25 02:15:00
End                       2024-10-02 01:05:00
Duration                      6 days 22:50:00
Exposure Time [%]                   82.975537
Equity Final [$]               1384874.466398
Equity Peak [$]                1441084.181271
Return [%]                          38.487447
Buy & Hold Return [%]                3.223338
Return (Ann.) [%]            283041236.914447
Volatility (Ann.) [%]        547115731.102736
Sharpe Ratio                         0.517333
Sortino Ratio                 40979381.143414
Calmar Ratio                  54437058.882588
Max. Drawdown [%]                   -5.199422
Avg. Drawdown [%]                   -1.329384
Max. Drawdown Duration        1 days 18:55:00
Avg. Drawdown Duration        0 days 04:33:00
# Trades                                   49
Win Rate [%]                        34.693878
Best Trade [%]                      17.232464
Worst Trade [%]                      -1.07677
Avg. Trade [%]                    

In [25]:
# plot
tester.plot(show_price_data=False)
plt.show()